In [41]:
import pandas as pd
from recipe_scrapers import scrape_me
import time
import random
import json
import pymongo
from tqdm.notebook import tqdm
import numpy as np

In [3]:
def wait():
    
    x = random.randrange(50, 200, 1)/100
    print(f"Waiting for {x} Seconds.", end='\r')
    time.sleep(x)

In [4]:
# Db Information
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['food_analysis']
urls = db['urls']
recipes = db['recipes']

# Scraper
Take the url database and start retrieving the actual recipe contents to store in a new database dedicated to the recipes. Given that this database will form the basis for an actual app and analysis the key unique id will be generated by Mongo and keep close tabs with the url database to ensure no read recipes are reread.

In [ ]:
eligible_urls = [x["_id"] for x in list(urls.find({"read":False}, {"_id":1}))]
eligible_urls = sorted(eligible_urls, key = lambda x: random.random())

for url in tqdm(eligible_urls):
    
    # Obtain existing data
    print(url)
    row = list(urls.find({"_id":url}))[0]
    
    
    try:
        if url[0] != 'h':
            url = "https:" + url

        # Pause and then scrape
        wait()
        scraper = scrape_me(url)

        # Attempt each piece of data

        # Title
        try:
            title = scraper.title()
        except:
            try:
                title = row['name']
            except:
                title = None

        # Total Time
        try:
            total_time = scraper.total_time()
        except:
            total_time = None

        # Yields
        try:
            yields = scraper.yields()
        except:
            yields = None

        # Ingredients
        try:
            ingredients = scraper.ingredients()
        except:
            ingredients = []

        # Instructions
        try:
            instructions = scraper.instructions().encode('utf-8', 'surrogatepass')
        except:
            instructions = None

        # Image
        try:
            image = scraper.image()
        except:
            image = None

        # Ratings
        try:
            rating = scraper.ratings()
        except:
            rating = None

        # Author
        try:
            author = scraper.author()
        except:
            try:
                author = row['author']
            except:
                author = None

        # Reviews
        try:
            reviews = scraper.reviews()
        except:
            reviews = None

        # Insert new data
        recipes.insert_one({"title":title,
                            "total_time":total_time,
                            "yields":yields,
                            "ingredients":ingredients,
                            "instructions":instructions,
                            "image":image,
                            "rating":rating,
                            "author":author,
                            "reviews":reviews,
                            "source":row['source'],
                            "url":row["_id"]})

        # Mark as read
        query = {"_id":url}
        newvalues = {"$set":{"read":True}}
        urls.update_one(query, newvalues)
        
    except:
        
        # Mark as Error
        print("Error encountered.")
        query = {"_id":url}
        newvalues = {"$set":{"error":True}}
        urls.update_one(query, newvalues)

//www.foodnetwork.com/recipes/alton-brown/scones-recipe-1909774
https://www.epicurious.com/recipes/food/views/mother-sherman-200346
https://www.food.com/recipe/krispy-kreme-bread-pudding-i-94463
https://www.epicurious.com/recipes/food/views/easy-pad-thai-51222610
//www.foodnetwork.com/recipes/giada-de-laurentiis/persimmon-pumpkin-pie-recipe-2126574
https://www.food.com/recipe/strawberry-country-cake-14432
https://www.food.com/recipe/chocolate-brownie-diabetic-210343
//www.foodnetwork.com/recipes/ina-garten/mini-italian-frittatas-4720584
https://www.mybakingaddiction.com/chipster-topped-brownies/
https://www.food.com/recipe/peanut-butter-pinwheel-cookies-333655
https://www.epicurious.com/recipes/food/views/homemade-flour-tortillas-233305
https://www.food.com/recipe/lazy-banana-muffins-in-the-oven-in-just-over-5-minutes-155922
https://www.food.com/recipe/chocolate-pudding-342689
//www.foodnetwork.com/recipes/ina-garten/corn-muffins-recipe-1948484
https://www.epicurious.com/recipes/food/v

https://www.food.com/recipe/bean-soup-with-pasta-water-528384
https://www.food.com/recipe/cranberry-sausage-and-apple-stuffing-264246
https://www.epicurious.com/recipes/food/views/grilled-apricots-with-almond-cream-and-fregolotta
https://www.epicurious.com/recipes/food/views/chili-potato-soup-20101
https://www.closetcooking.com/zucchini-turkey-meatballs-in-marinara-on-zoodles/
https://www.food.com/recipe/chicken-enchiladas-465928
https://www.food.com/recipe/banana-split-fruit-salad-341272
https://www.food.com/recipe/sweet-basil-carrots-54634
https://www.epicurious.com/recipes/food/views/herb-marinated-feta-and-olives-105650
https://www.food.com/recipe/copycat-pace-picante-sauce-81059
https://www.epicurious.com/recipes/food/views/spring-greens-with-smoked-fish-and-herbed-aioli-364669
//www.foodnetwork.com/recipes/trisha-yearwood/caramelized-onion-dip-3229685
https://www.food.com/recipe/indonesian-cucumber-salad-354950
https://www.food.com/recipe/easy-celebration-ice-cream-cake-366786
ht

https://www.food.com/recipe/hamburger-soup-270680
//www.foodnetwork.com/recipes/sandra-lee/artichoke-and-black-bean-nachos-recipe-1915716
https://www.food.com/recipe/flourless-chocolate-truffle-cake-62920
//www.foodnetwork.com/recipes/aarti-sequeira/vanilla-and-black-pepper-pork-loin-recipe-1922853
https://www.food.com/recipe/susies-party-cheese-ball-94938
//www.foodnetwork.com/recipes/ree-drummond/pecan-pie-recipe4-1922479
https://www.food.com/recipe/leek-fritters-prassokeftedes-141973
https://www.food.com/recipe/karens-corned-beef-hash-sandwich-242894
https://www.food.com/recipe/italian-chicken-438678
https://www.food.com/recipe/fabulous-cheesecake-with-blueberry-glaze-216600
https://www.epicurious.com/recipes/food/views/oxtail-pate-11072
https://www.food.com/recipe/chicken-and-rice-bake-320042
https://www.food.com/recipe/chocolate-kahlua-toffee-trifle-33258
https://www.epicurious.com/recipes/food/views/raspberry-topped-lemon-muffins-235329
https://www.food.com/recipe/beef-and-brocco

https://www.epicurious.com/recipes/food/views/mixed-mushroom-and-tarragon-gravy-361769
https://www.food.com/recipe/peanut-butter-and-banana-smoothie-382202
https://www.food.com/recipe/japanese-vegetable-soup-with-chicken-kenchin-jiru-514482
https://www.cookstr.com/recipes/classic-smores
https://www.food.com/recipe/cream-cheese-bundt-cake-304179
https://www.epicurious.com/recipes/food/views/barbecue-chicken-wings-241549
https://www.food.com/recipe/true-lemon-chicken-51884
https://www.food.com/recipe/asian-salad-55980
https://www.food.com/recipe/chicken-pasta-with-marinara-sauce-103704
https://www.food.com/recipe/turkey-cranberry-quesadillas-49474
https://www.epicurious.com/recipes/food/views/potato-gratin-with-goat-cheese-51259670
https://www.southernliving.com/recipes/easy-southern-fried-chicken
https://www.epicurious.com/recipes/food/views/strawberry-gin-yogurt-cocktail-slippery-when-wet
https://www.epicurious.com/recipes/food/views/water-spinach-with-shrimp-234663
https://www.food.co

https://www.food.com/recipe/ciro-and-sals-salad-dressing-97432
https://www.food.com/recipe/carolina-style-barbecue-sauce-31766
https://www.food.com/recipe/soft-fried-shanghai-spicy-noodles-with-shredded-pork-83056
https://www.food.com/recipe/sugar-drop-cookies-273272
https://www.epicurious.com/recipes/food/views/baked-pears-with-currants-and-cinnamon-102884
https://www.food.com/recipe/beckys-cream-style-corn-102185
https://www.food.com/recipe/enchilada-sauce-17146
//www.foodnetwork.com/recipes/michael-chiarello/s-mores-at-the-door-recipe-2013832
https://www.food.com/recipe/crock-pot-gingerbread-pudding-cake-295725
https://www.food.com/recipe/georgian-beets-in-cherry-sauce-charkhalis-chogi-307308
https://www.food.com/recipe/spicy-cocktail-nuts-108560
https://www.fifteenspatulas.com/freezer-friendly-shepherds-pie/
//www.foodnetwork.com/recipes/michael-chiarello/spiked-cider-cocktail-recipe-2014318
https://www.food.com/recipe/gluten-free-chicken-pot-pie-335417
https://www.food.com/recipe/

...

https://www.food.com/recipe/cinnamon-streusel-apple-cake-513560
https://www.epicurious.com/recipes/food/views/sweet-mustard-sauce-234999
https://www.food.com/recipe/singapore-chicken-stew-187041
https://www.food.com/recipe/delicious-turkey-tetrazzini-291479
https://www.epicurious.com/recipes/food/views/shrimp-and-pancetta-on-polenta-356050
https://www.epicurious.com/recipes/food/views/buckwheat-linzer-cookies-51257390
https://www.food.com/recipe/refreshing-lemon-asparagus-218808
https://www.food.com/recipe/lemon-dijon-asparagus-61848
https://www.epicurious.com/recipes/food/views/apricot-cranberry-and-walnut-pie-102542
https://www.epicurious.com/recipes/food/views/lettuce-soup-101521
https://www.food.com/recipe/oatmeal-apple-cranberry-scones-118506
https://www.food.com/recipe/maple-glazed-bacon-148294
https://www.food.com/recipe/oyster-shooter-adult-beverage-157691
https://www.epicurious.com/recipes/food/views/baba-105879
https://www.food.com/recipe/italian-meat-sauce-431424
https://www

//www.foodnetwork.com/recipes/sunny-anderson/faux-fongo-with-red-bean-gravy-recipe-2014690
https://www.food.com/recipe/dilled-carrot-sticks-49071
https://www.cookstr.com/recipes/orange-chocolate-mousse-parfait
https://www.cookstr.com/recipes/olivada
https://www.food.com/recipe/slow-cooker-cream-cheese-chicken-chili-472700
https://www.epicurious.com/recipes/food/views/peppermint-cream-puff-ring-with-chocolate-glaze-233427
https://www.food.com/recipe/grilled-gouda-cheese-sandwiches-with-smoked-ham-and-pumpernickel-426031
https://www.epicurious.com/recipes/food/views/steamed-new-potatoes-with-mint-and-parsley-11872
https://www.food.com/recipe/roasted-garlic-mashed-potatoes-lower-healthier-fat-367672
https://www.food.com/recipe/chilled-spicy-seafood-sauce-33592
//www.foodnetwork.com/recipes/aida-mollenkamp/three-cheese-quesadillas-recipe-1957291
https://www.food.com/recipe/enchiladas-with-red-sauce-603
https://www.closetcooking.com/mozzarella-baked-gnocchi-bolognese/
https://www.food.com/r

https://www.food.com/recipe/smoked-cheddar-quesadillas-86450
https://www.epicurious.com/recipes/food/views/truffled-toast-with-radicchio-and-egg-236003
https://www.food.com/recipe/cheeseburger-zucchini-pie-322544
https://www.food.com/recipe/back-uh-the-can-hawaiian-chicken-291834
https://www.food.com/recipe/speckkartoffel-potatoes-with-bacon-69650
https://www.closetcooking.com/balsamic-honey-and-mustard-pork-chops/
https://www.food.com/recipe/bbq-bacon-chicken-76414
https://www.food.com/recipe/miso-glazed-chicken-wings-with-a-ginger-scallion-dipping-487007
https://www.southernliving.com/syndication/collard-greens-with-black-eyed-peas
//www.foodnetwork.com/recipes/katie-lee/apple-orange-cranberry-relish-2663542
https://www.food.com/recipe/potato-pancakes-470452
https://www.southernliving.com/syndication/heavenly-angel-food-cake
https://www.food.com/recipe/apple-pie-spirits-202658
https://www.food.com/recipe/white-bean-soup-with-andouille-and-collards-479701
https://www.epicurious.com/re

https://www.epicurious.com/recipes/food/views/napa-cabbage-kimchi-104681
https://www.food.com/recipe/an-arabic-breakfast-90316
https://www.food.com/recipe/p-f-changs-shrimp-with-lobster-sauce-512305
https://www.food.com/recipe/eggplant-rolls-w-tomato-sauce-220002
https://www.food.com/recipe/apple-city-barbecue-sauce-mike-mills-428306
https://www.food.com/recipe/best-ever-three-bean-chili-348339
https://www.food.com/recipe/pear-and-prosciutto-di-parma-salad-66685
https://www.food.com/recipe/pecan-caramel-candies-145177
https://www.food.com/recipe/matthews-chicken-curry-492854
https://www.food.com/recipe/apple-doughnut-muffins-104648
https://www.epicurious.com/recipes/food/views/dark-roast-coffee-gelee-109234
https://www.food.com/recipe/shamrock-casserole-130820
https://www.epicurious.com/recipes/food/views/forked-oven-roasted-potatoes-368937
https://www.food.com/recipe/crock-pot-orange-chicken-484175
//www.foodnetwork.com/recipes/rachael-ray/eggplant-and-squash-curry-recipe-2200988
//ww

https://www.food.com/recipe/potato-omelet-torta-de-papas-170591
https://www.food.com/recipe/holiday-ham-with-herb-crust-and-brown-sugar-cider-glaze-511692
https://www.food.com/recipe/capri-salad-52224
https://www.food.com/recipe/eggplant-tomato-and-green-onion-curry-253219
https://www.food.com/recipe/bunny-bread-w-dip-in-tummy-163034
https://www.food.com/recipe/salmon-with-sweet-chili-glaze-452563
https://www.food.com/recipe/mealworm-fried-rice-449381
https://www.mybakingaddiction.com/almond-butter-cookies/
https://www.food.com/recipe/sweet-potato-casserole-155004
https://www.food.com/recipe/mango-chutney-chicken-274743
https://www.food.com/recipe/curry-citrus-chicken-154747
https://www.food.com/recipe/kom-tang-korean-beef-stock-soup-488688
https://www.food.com/recipe/chinese-lo-mein-with-peanut-butter-sauce-288077
https://www.food.com/recipe/greek-mushroom-salad-544
https://www.epicurious.com/recipes/food/views/goat-cheese-mashed-potatoes-with-leeks-and-chives-12790
https://www.food.c

# Database Cleaning and Maintenance

In [67]:
# Print Overview
total = len(list(urls.find({})))
read = len(list(urls.find({"read":True})))
togo = len(list(urls.find({"read":False})))
errors = len(list(urls.find({"error":True})))
in_recipes = len(list(recipes.find({})))

print(f"URL Database: {total}")
print(f"Read: {read}")
print(f"To Read: {togo}")
print(f"Errors: {errors}")
print(f"Recipes Database: {in_recipes}")

URL Database: 220842
Read: 29245
To Read: 191597
Errors: 20
Recipes Database: 29245


### Duplicate Cleaning
At this stage should be true duplicates, if urls are found more than once it keeps first, which is arbitrary

In [53]:
# Remove Duplicates (true dups, everything is identical)
dups = pd.DataFrame(list(recipes.find({})))
dups = dups[["_id", "url"]].groupby("url", as_index=False).count()
dups = np.unique(dups[dups["_id"]>1]["url"])
for url in dups:
    delete = [x["_id"] for x in list(recipes.find({"url":url}))[1:]]
    for did in delete:
        recipes.delete_one({"_id":did})

### Marking Read
Depending on certai issues some documents don't get marked as read. If they are in recipes then they should be updated and this quick fix will clean that up

In [66]:
# Mark everything as read
read_urls = [x["url"] for x in list(recipes.find({}))]
for url in tqdm(read_urls):
    urls.update_one({"_id":url}, {"$set":{"read":True, "error":False}})